# Macquarie VRP Analysis
This notebok runs analysis on Macquaries VRP strategies.

## Import Libraries

In [ ]:
import os
os.chdir('..')

import plotly.express as px
from EquityHedging.datamanager import data_manager as dm
from EquityHedging.analytics import summary
from EquityHedging.reporting.excel import reports as rp
from EquityHedging.reporting import formatter as fmt, plots
from ipywidgets import interact, interact_manual
import pandas as pd
import plotly.graph_objects as go
corr_freq_list = ['Weekly', 'Monthly']
analytics_freq_list = ['Weekly', 'Monthly']

## Import Equity Benchmark & UBS Def Var Data

In [ ]:
equity_bmk = 'SPTR'
#create returns data dictionary for equity benchmark
returns= dm.get_equity_hedge_returns(equity_bmk, all_data=True)
for key in returns:
    returns[key] = returns[key][[equity_bmk,'Def Var (Fri)', 'Def Var (Mon)','Def Var (Wed)']]

# VRP + TRP Analysis
Strats:
* VMAQWSP4
* VMAQDSP4
* VMAQWSP4 extended + Deferred TRP (“always on”)
* VMAQWSP4 extended + Front TRP (subject to VIX Signal)
* VMAQWSP4 extended + Combined TRP (Deferred + Front with dynamic VIX Signal)
* VMAQDSP4 extended + VMAQWSP4 extended + Deferred TRP (“always on”)
* VMAQDSP4 extended + VMAQWSP4 extended + Front TRP (subject to VIX Signal)
* VMAQDSP4 extended + VMAQWSP4 extended + Combined TRP (Deferred + Front with dynamic VIX Signal)

## Import data

In [ ]:
trp = dm.get_new_strategy_returns_data('vrp_macq_data.xlsx', sheet_name='trp')
#create returns data dictionary for strategy
trp_dict = dm.get_data_dict(trp)
#merge dictionaries
trp_dict = dm.merge_dicts(returns,trp_dict)

## Display Historical Performance

In [ ]:
@interact
def display_performance():
    fig = go.Figure()
    for strat in trp.columns:
        fig.add_trace(go.Scatter(x = pd.to_datetime(trp.index, format = '%m/%d/%Y'),
                                 y = trp[strat],mode = 'lines',name = strat))
    fig.update_layout(showlegend=True,plot_bgcolor='White')
    fig.update_xaxes(showline=True,linecolor='black')
    fig.update_yaxes(showline=True,linecolor='black')
    fig.show()

## Compute and Display TRP Correlations

In [ ]:
trp_corr_dict = summary.get_corr_data(trp_dict)
@interact
def display_correlations(frequency=corr_freq_list,
                         corr=['full', 'equity_down', 'equity_up'], plot_type=['heatmap', 'corrplot']):
    data = trp_corr_dict[frequency]
    corr_df = data[False][corr][0]
    if plot_type == 'heatmap':
        return plots.draw_heatmap(corr_df, half=False)
    else:
        return plots.draw_corrplot(corr_df)

## Compute and Display TRP Analytics (Return Stats and Hedge Metrics)

In [ ]:
trp_analytics_dict = summary.get_analytics_data(trp_dict)
@interact
def display_analytics(frequency=analytics_freq_list, stats=['return_stats', 'hedge_metrics']):
    data = trp_analytics_dict[frequency]
    analytics = data[False]
    if stats == 'return_stats':
        return fmt.get_analytics_styler(analytics)
    if stats == 'hedge_metrics':
        if frequency == 'Weekly':
            return fmt.get_analytics_styler(analytics, stats, '1W')
        else:
            return fmt.get_analytics_styler(analytics, stats)

## Compute and Display Historical Selloffs

In [ ]:
trp_hist_dict = summary.get_hist_data(trp_dict)
@interact
def display_selloffs():
    df_hist = trp_hist_dict[False]
    return fmt.get_hist_styler(df_hist)

# VRP (VMAQWSP4 – with/OUT defensive signal) + Long/Short Intraday Basket Analysis
Strats:
* VMAQWSP4
* VMAQWSP4 + Weight of Long/Short Basket ( 0%, 5%, 10..., 95%, 100%)

## Import data

In [ ]:
wsp4_eib = dm.get_new_strategy_returns_data('vrp_macq_data.xlsx', sheet_name='wsp4_eib')
#create returns data dictionary for strategy
wsp4_eib_dict = dm.get_data_dict(wsp4_eib)
#merge dictionaries
wsp4_eib_dict = dm.merge_dicts(returns,wsp4_eib_dict)

## Display Historical Performance

In [ ]:
@interact
def display_performance():
    fig = go.Figure()
    for strat in wsp4_eib.columns:
        fig.add_trace(go.Scatter(x = pd.to_datetime(wsp4_eib.index, format = '%m/%d/%Y'),
                                 y = wsp4_eib[strat],mode = 'lines',name = strat))
    fig.update_layout(showlegend=True,plot_bgcolor='White')
    fig.update_xaxes(showline=True,linecolor='black')
    fig.update_yaxes(showline=True,linecolor='black')
    fig.show()

## Compute and Display Correlations

In [ ]:
wsp4_eib_corr_dict = summary.get_corr_data(wsp4_eib_dict)
@interact
def display_correlations(frequency=corr_freq_list,
                         corr=['full', 'equity_down', 'equity_up'], plot_type=['heatmap', 'corrplot']):
    data = wsp4_eib_corr_dict[frequency]
    corr_df = data[False][corr][0]
    if plot_type == 'heatmap':
        return plots.draw_heatmap(corr_df, half=False)
    else:
        return plots.draw_corrplot(corr_df)

## Compute and Display Analytics (Return Stats and Hedge Metrics)

In [ ]:
wsp4_eib_analytics_dict = summary.get_analytics_data(wsp4_eib_dict)
@interact
def display_analytics(frequency=analytics_freq_list, stats=['return_stats', 'hedge_metrics']):
    data = wsp4_eib_analytics_dict[frequency]
    analytics = data[False]
    if stats == 'return_stats':
        return fmt.get_analytics_styler(analytics)
    if stats == 'hedge_metrics':
        if frequency == 'Weekly':
            return fmt.get_analytics_styler(analytics, stats, '1W')
        else:
            return fmt.get_analytics_styler(analytics, stats)

## Compute and Display Historical Selloffs

In [ ]:
wsp4_eib_hist_dict = summary.get_hist_data(wsp4_eib_dict)
@interact
def display_selloffs():
    df_hist = wsp4_eib_hist_dict[False]
    return fmt.get_hist_styler(df_hist)

# VRP (VMAQWSP4 – with/OUT defensive signal) + Short Intraday Basket Analysis
Strats:
* VMAQWSP4
* VMAQWSP4 + Weight of Short Basket ( 0%, 5%, 10..., 95%, 100%)

## Import data

In [ ]:
wsp4_eib_s = dm.get_new_strategy_returns_data('vrp_macq_data.xlsx', sheet_name='wsp4_eib_short')
#create returns data dictionary for strategy
wsp4_eib_s_dict = dm.get_data_dict(wsp4_eib_s)
#merge dictionaries
wsp4_eib_s_dict = dm.merge_dicts(returns,wsp4_eib_s_dict)

## Display Historical Performance

In [ ]:
@interact
def display_performance():
    fig = go.Figure()
    for strat in wsp4_eib_s.columns:
        fig.add_trace(go.Scatter(x = pd.to_datetime(wsp4_eib_s.index, format = '%m/%d/%Y'),
                                 y = wsp4_eib_s[strat],mode = 'lines',name = strat))
    fig.update_layout(showlegend=True,plot_bgcolor='White')
    fig.update_xaxes(showline=True,linecolor='black')
    fig.update_yaxes(showline=True,linecolor='black')
    fig.show()

## Compute and Display Correlations

In [ ]:
wsp4_eib_s_corr_dict = summary.get_corr_data(wsp4_eib_s_dict)
@interact
def display_correlations(frequency=corr_freq_list,
                         corr=['full', 'equity_down', 'equity_up'], plot_type=['heatmap', 'corrplot']):
    data = wsp4_eib_s_corr_dict[frequency]
    corr_df = data[False][corr][0]
    if plot_type == 'heatmap':
        return plots.draw_heatmap(corr_df, half=False)
    else:
        return plots.draw_corrplot(corr_df)

## Compute and Display Analytics (Return Stats and Hedge Metrics)

In [ ]:
wsp4_eib_s_analytics_dict = summary.get_analytics_data(wsp4_eib_s_dict)
@interact
def display_analytics(frequency=analytics_freq_list, stats=['return_stats', 'hedge_metrics']):
    data = wsp4_eib_s_analytics_dict[frequency]
    analytics = data[False]
    if stats == 'return_stats':
        return fmt.get_analytics_styler(analytics)
    if stats == 'hedge_metrics':
        if frequency == 'Weekly':
            return fmt.get_analytics_styler(analytics, stats, '1W')
        else:
            return fmt.get_analytics_styler(analytics, stats)

## Compute and Display Historical Selloffs

In [ ]:
wsp4_eib_s_hist_dict = summary.get_hist_data(wsp4_eib_s_dict)
@interact
def display_selloffs():
    df_hist = wsp4_eib_s_hist_dict[False]
    return fmt.get_hist_styler(df_hist)

# VRP (DSP4 – with/OUT defensive signal) + Long/Short Intraday Basket Analysis
Strats:
* VMAQDSP4
* VMAQDSP4 + Weight of Long/Short Basket ( 0%, 5%, 10..., 95%, 100%)

## Import data

In [ ]:
dsp4_eib = dm.get_new_strategy_returns_data('vrp_macq_data.xlsx', sheet_name='dsp4_eib')
#create returns data dictionary for strategy
dsp4_eib_dict = dm.get_data_dict(dsp4_eib)
#merge dictionaries
dsp4_eib_dict = dm.merge_dicts(returns,dsp4_eib_dict)

## Display Historical Performance

In [ ]:
@interact
def display_performance():
    fig = go.Figure()
    for strat in dsp4_eib.columns:
        fig.add_trace(go.Scatter(x = pd.to_datetime(dsp4_eib.index, format = '%m/%d/%Y'),
                                 y = dsp4_eib[strat],mode = 'lines',name = strat))
    fig.update_layout(showlegend=True,plot_bgcolor='White')
    fig.update_xaxes(showline=True,linecolor='black')
    fig.update_yaxes(showline=True,linecolor='black')
    fig.show()

## Compute and Display Correlations

In [ ]:
dsp4_eib_corr_dict = summary.get_corr_data(dsp4_eib_dict)
@interact
def display_correlations(frequency=corr_freq_list,
                         corr=['full', 'equity_down', 'equity_up'], plot_type=['heatmap', 'corrplot']):
    data = dsp4_eib_corr_dict[frequency]
    corr_df = data[False][corr][0]
    if plot_type == 'heatmap':
        return plots.draw_heatmap(corr_df, half=False)
    else:
        return plots.draw_corrplot(corr_df)

## Compute and Display Analytics (Return Stats and Hedge Metrics)

In [ ]:
dsp4_eib_analytics_dict = summary.get_analytics_data(dsp4_eib_dict)
@interact
def display_analytics(frequency=analytics_freq_list, stats=['return_stats', 'hedge_metrics']):
    data = dsp4_eib_analytics_dict[frequency]
    analytics = data[False]
    if stats == 'return_stats':
        return fmt.get_analytics_styler(analytics)
    if stats == 'hedge_metrics':
        if frequency == 'Weekly':
            return fmt.get_analytics_styler(analytics, stats, '1W')
        else:
            return fmt.get_analytics_styler(analytics, stats)

## Compute and Display Historical Selloffs

In [ ]:
dsp4_eib_hist_dict = summary.get_hist_data(dsp4_eib_dict)
@interact
def display_selloffs():
    df_hist = dsp4_eib_hist_dict[False]
    return fmt.get_hist_styler(df_hist)

# VRP (DSP4 – with/OUT defensive signal) + Short Intraday Basket Analysis
Strats:
* VMAQDSP4
* VMAQDSP4 + Weight of Short Basket ( 0%, 5%, 10..., 95%, 100%)

## Import data

In [ ]:
dsp4_eib_s = dm.get_new_strategy_returns_data('vrp_macq_data.xlsx', sheet_name='dsp4_eib_short')
#create returns data dictionary for strategy
dsp4_eib_s_dict = dm.get_data_dict(dsp4_eib_s)
#merge dictionaries
dsp4_eib_s_dict = dm.merge_dicts(returns,dsp4_eib_s_dict)

## Display Historical Performance

In [ ]:
@interact
def display_performance():
    fig = go.Figure()
    for strat in dsp4_eib_s.columns:
        fig.add_trace(go.Scatter(x = pd.to_datetime(dsp4_eib_s.index, format = '%m/%d/%Y'),
                                 y = dsp4_eib_s[strat],mode = 'lines',name = strat))
    fig.update_layout(showlegend=True,plot_bgcolor='White')
    fig.update_xaxes(showline=True,linecolor='black')
    fig.update_yaxes(showline=True,linecolor='black')
    fig.show()

## Compute and Display Correlations

In [ ]:
dsp4_eib_s_corr_dict = summary.get_corr_data(dsp4_eib_s_dict)
@interact
def display_correlations(frequency=corr_freq_list,
                         corr=['full', 'equity_down', 'equity_up'], plot_type=['heatmap', 'corrplot']):
    data = dsp4_eib_s_corr_dict[frequency]
    corr_df = data[False][corr][0]
    if plot_type == 'heatmap':
        return plots.draw_heatmap(corr_df, half=False)
    else:
        return plots.draw_corrplot(corr_df)

## Compute and Display Analytics (Return Stats and Hedge Metrics)

In [ ]:
dsp4_eib_s_analytics_dict = summary.get_analytics_data(dsp4_eib_s_dict)
@interact
def display_analytics(frequency=analytics_freq_list, stats=['return_stats', 'hedge_metrics']):
    data = dsp4_eib_s_analytics_dict[frequency]
    analytics = data[False]
    if stats == 'return_stats':
        return fmt.get_analytics_styler(analytics)
    if stats == 'hedge_metrics':
        if frequency == 'Weekly':
            return fmt.get_analytics_styler(analytics, stats, '1W')
        else:
            return fmt.get_analytics_styler(analytics, stats)

## Compute and Display Historical Selloffs

In [ ]:
dsp4_eib_s_hist_dict = summary.get_hist_data(dsp4_eib_s_dict)
@interact
def display_selloffs():
    df_hist = dsp4_eib_s_hist_dict[False]
    return fmt.get_hist_styler(df_hist)

# VRP (WSP4 Fri-pre2017 – with/OUT defensive signal) + Long/Short Intraday Basket Analysis
Strats:
* VMAQWSP4 Fri-pre 2017
* VMAQWSP4 Fri-pre 2017 + Weight of Long/Short Basket ( 0%, 5%, 10..., 95%, 100%)

## Import data

In [ ]:
wsp4_eib_pre17 = dm.get_new_strategy_returns_data('vrp_macq_data.xlsx', sheet_name='wsp4_eib_pre17')
#create returns data dictionary for strategy
wsp4_eib_pre17_dict = dm.get_data_dict(wsp4_eib_pre17)
#merge dictionaries
wsp4_eib_pre17_dict = dm.merge_dicts(returns,wsp4_eib_pre17_dict)

## Display Historical Performance

In [ ]:
@interact
def display_performance():
    fig = go.Figure()
    for strat in wsp4_eib_pre17.columns:
        fig.add_trace(go.Scatter(x = pd.to_datetime(wsp4_eib_pre17.index, format = '%m/%d/%Y'),
                                 y = wsp4_eib_pre17[strat],mode = 'lines',name = strat))
    fig.update_layout(showlegend=True,plot_bgcolor='White')
    fig.update_xaxes(showline=True,linecolor='black')
    fig.update_yaxes(showline=True,linecolor='black')
    fig.show()

## Compute and Display Correlations

In [ ]:
wsp4_eib_pre17_corr_dict = summary.get_corr_data(wsp4_eib_pre17_dict)
@interact
def display_correlations(frequency=corr_freq_list,
                         corr=['full', 'equity_down', 'equity_up'], plot_type=['heatmap', 'corrplot']):
    data = wsp4_eib_pre17_corr_dict[frequency]
    corr_df = data[False][corr][0]
    if plot_type == 'heatmap':
        return plots.draw_heatmap(corr_df, half=False)
    else:
        return plots.draw_corrplot(corr_df)

## Compute and Display Analytics (Return Stats and Hedge Metrics)

In [ ]:
wsp4_eib_pre17_analytics_dict = summary.get_analytics_data(wsp4_eib_pre17_dict)
@interact
def display_analytics(frequency=analytics_freq_list, stats=['return_stats', 'hedge_metrics']):
    data = wsp4_eib_pre17_analytics_dict[frequency]
    analytics = data[False]
    if stats == 'return_stats':
        return fmt.get_analytics_styler(analytics)
    if stats == 'hedge_metrics':
        if frequency == 'Weekly':
            return fmt.get_analytics_styler(analytics, stats, '1W')
        else:
            return fmt.get_analytics_styler(analytics, stats)

## Compute and Display Historical Selloffs

In [ ]:
wsp4_eib_pre17_hist_dict = summary.get_hist_data(wsp4_eib_pre17_dict)
@interact
def display_selloffs():
    df_hist = wsp4_eib_pre17_hist_dict[False]
    return fmt.get_hist_styler(df_hist)

# VRP (WSP4 Fri-pre2017 – with/OUT defensive signal) + Short Intraday Basket Analysis
Strats:
* VMAQWSP4 Fri-pre2017
* VMAQWSP4 Fri-pre2017 + Weight of Short Basket ( 0%, 5%, 10..., 95%, 100%)

## Import data

In [ ]:
wsp4_eib_pre17_s = dm.get_new_strategy_returns_data('vrp_macq_data.xlsx', sheet_name='wsp4_eib_pre17_short')
#create returns data dictionary for strategy
wsp4_eib_pre17_s_dict = dm.get_data_dict(wsp4_eib_pre17_s)
#merge dictionaries
wsp4_eib_pre17_s_dict = dm.merge_dicts(returns,wsp4_eib_pre17_s_dict)

## Display Historical Performance

In [ ]:
@interact
def display_performance():
    fig = go.Figure()
    for strat in wsp4_eib_pre17_s.columns:
        fig.add_trace(go.Scatter(x = pd.to_datetime(wsp4_eib_pre17_s.index, format = '%m/%d/%Y'),
                                 y = wsp4_eib_pre17_s[strat],mode = 'lines',name = strat))
    fig.update_layout(showlegend=True,plot_bgcolor='White')
    fig.update_xaxes(showline=True,linecolor='black')
    fig.update_yaxes(showline=True,linecolor='black')
    fig.show()

## Compute and Display Correlations

In [ ]:
wsp4_eib_pre17_s_corr_dict = summary.get_corr_data(wsp4_eib_pre17_s_dict)
@interact
def display_correlations(frequency=corr_freq_list,
                         corr=['full', 'equity_down', 'equity_up'], plot_type=['heatmap', 'corrplot']):
    data = wsp4_eib_pre17_s_corr_dict[frequency]
    corr_df = data[False][corr][0]
    if plot_type == 'heatmap':
        return plots.draw_heatmap(corr_df, half=False)
    else:
        return plots.draw_corrplot(corr_df)

## Compute and Display Analytics (Return Stats and Hedge Metrics)

In [ ]:
wsp4_eib_pre17_s_analytics_dict = summary.get_analytics_data(wsp4_eib_pre17_s_dict)
@interact
def display_analytics(frequency=analytics_freq_list, stats=['return_stats', 'hedge_metrics']):
    data = wsp4_eib_pre17_s_analytics_dict[frequency]
    analytics = data[False]
    if stats == 'return_stats':
        return fmt.get_analytics_styler(analytics)
    if stats == 'hedge_metrics':
        if frequency == 'Weekly':
            return fmt.get_analytics_styler(analytics, stats, '1W')
        else:
            return fmt.get_analytics_styler(analytics, stats)

## Compute and Display Historical Selloffs

In [ ]:
wsp4_eib_pre17_s_hist_dict = summary.get_hist_data(wsp4_eib_pre17_s_dict)
@interact
def display_selloffs():
    df_hist = wsp4_eib_pre17_s_hist_dict[False]
    return fmt.get_hist_styler(df_hist)

# Create Report

Run this code below to export analysis into excel spreadsheet
* **strat_report (string)**: provide a name for the excel file
* **selloffs (boolean)**:
    * **True**: if you want historical selloff data in the spreadsheet
    * **False**: if you do not want historical selloff data in the spreadsheet

In [ ]:
strats_dict = {'trp':trp_dict,'wsp4_eib':wsp4_eib_dict,'wsp4_eib_s':wsp4_eib_s_dict,'dsp4_eib':dsp4_eib_dict,
               'dsp4_eib_s':dsp4_eib_s_dict,'wsp4_eib_pre17':wsp4_eib_pre17_dict,'wsp4_eib_pre17_s':wsp4_eib_pre17_s_dict}
selloffs = True
for key in strats_dict:
    rp.generate_strat_report(key, strats_dict[key], selloffs)